In [1]:
import graphlab as gl
import matplotlib.pyplot as plt
import os
import graphlab.aggregate as agg

In [6]:
def Cluster_Volume(filename):    
    def Edges(P):
        points = sorted(set(P))
        def cross(o, a, b):
            return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
        lower = []
        for count in points:
                while len(lower) >= 2 and cross(lower[-2], lower[-1], count) <= 0:
                    lower.pop()
                lower.append(count)
        upper = []
        for count in reversed(points):
                while len(upper) >= 2 and cross(upper[-2], upper[-1], count) <= 0:
                    upper.pop()
                upper.append(count)
        shape = lower[:-1] + upper[:-1]
        return shape

    def Area(shape):
        Area = 0
        Add = 0
        Sub = 0
        for a in range(len(shape)):
            if a != (len(shape)-1):
                Add = Add + shape[a][0]* shape[a+1][1]
                Sub = Sub + shape[a][1]* shape[a+1][0]
            else:
                Add = Add + shape[a][0]* shape[0][1]
                Sub = Sub + shape[a][1]* shape[0][0]
        Area = (((Add - Sub)**2)**0.5)* 0.5
        return Area



 
    Raw_Image = gl.SFrame.read_csv(filename, delimiter = '	',header = True)

    #shift points so the min is (0,0)
    Sauce = gl.SFrame({'X': Raw_Image['Xc']-min(Raw_Image['Xc']),'Y': Raw_Image['Yc']-min(Raw_Image['Yc'])})
    
    #Runs DBSCAN on the points
    Image_C = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 20)
    Weak = gl.dbscan.create(Sauce, radius = 60, min_core_neighbors = 30)
    #Give each point its own identification number
    Sauce = Sauce.add_row_number('row_id')

    #Add the cluster identity to each point 
    Image_Pts = Sauce.join(Image_C['cluster_id'], on='row_id', how='left')
    Weak_s = Sauce.join(Weak['cluster_id'], on='row_id', how='left')
    core_mask = Image_Pts['type'] == 'core'
    boundary_mask = Image_Pts['type'] == 'boundary'
    plt.style.use('ggplot')
    #plt.axis([0, 5000, 0, 4000])
    plt.scatter(Sauce['X'],Sauce['Y'],s=5,c = 'black')
    plt.scatter(Image_Pts['X'][core_mask],Image_Pts['Y'][core_mask],s=5,c=Image_Pts['cluster_id'][core_mask],alpha = 0.9)
    plt.scatter(Image_Pts['X'][boundary_mask],Image_Pts['Y'][boundary_mask],s=5,c=Image_Pts['cluster_id'][boundary_mask],alpha = 0.5)
    
    
    #Image_Pts.show()
    
    
    Weak_s = Weak_s[Weak_s['cluster_id']>=0]
    #number of clusters
    #Num_C = (Image_Pts.sort('cluster_id', ascending = False))[0]['cluster_id']
    
    if (Image_Pts.sort('cluster_id', ascending = False))[0]['cluster_id'] == None:
        return #gl.SFrame({'cluster_id':[0],'Num_points':[0],
                #   'Clus_Vol':[0.0],
                 #  'Clus_Dens':[0.0],'Treatment':[Treat], 'File':[filename],
                  # 'Subject':[Subject],'% Tot Clus':[0],'Total Area':[0],
                   #'Total Clustered':[0]})
    else:
        Num_C = int((Image_Pts.sort('cluster_id', ascending = False))[0]['cluster_id'])
    
    Image_Pts = Image_Pts[Image_Pts['cluster_id']>=0] #Removes all non-clustered points
        
    #Results = gl.Sframe({})
    Area_t = 0
    if filename.find('++') > 0:
        Treat = 'Antagonist'
        #Treat = 'CytD'
    elif filename.find('+-')>0:
        Treat = 'KCl'
        #Treat = 'Nicotine'
        #Treat = 'mbCD'
        
    else :
        Treat = "Control"

    if filename.find('E')>0:
        Subject= 'Extension'
    elif filename.find('V')>0:
        Subject ='Varicosity' 
    else:
        Subject ='Soma'
        
    Clus_Vol = gl.SArray(data=[])
    
    for c in range(0,Num_C+1):
        cluster_mask = Image_Pts['cluster_id'] == c
        x,y = Image_Pts['X'][cluster_mask],Image_Pts['Y'][cluster_mask]
        P = zip(x,y)
        shape_c = Edges(P)
        shapex,shapey = zip(*shape_c)
        plt.scatter(shapex,shapey,edgecolors ='red',s=5)
        Area_c = Area(shape_c)
        Area_t = Area_c + Area_t
        qwe = gl.SArray(data=[Area_c])
        Clus_Vol = Clus_Vol.append(qwe)
        #print Area_c
        #print c
    

    x,y = Weak_s['X'],Weak_s['Y']
    P = zip(x,y)
    if len(P) >= 1:
        shape_V = Edges(P)
        shapex,shapey = zip(*shape_V)
        plt.scatter(shapex,shapey,s = 5, c ='orange')
        Area_Var = Area(shape_V)
    else:
        Area_Var = 0
    
    
    Total = Image_Pts.groupby(key_columns='cluster_id',operations={'Num_points':agg.COUNT('row_id')})
    Total = Total.sort('cluster_id')
    Total = Total.add_column(Clus_Vol,name ='Clus_Vol')
    Total = Total.add_column(Total['Num_points']/Total['Clus_Vol'],name='Clus_Dens')
    
    Total['Treatment']=Treat
    Total['Subject']=Subject
    Total['% Tot Clus']= Area_t / Area_Var
    Total['Total Area']= Area_Var
    Total['Total Clustered']= Area_t
    Total['File']=filename
    plt.savefig(filename +'.png', bbox_inches='tight')
    plt.close()
    return Total
    #print Area_Var
    
    #print Area_t/Area_Var
    
    #plt.show()




In [8]:
Total = gl.SFrame({'cluster_id':gl.SArray(dtype = int),'Num_points':gl.SArray(dtype = int),
                   'Clus_Vol':gl.SArray(dtype = float),
                   'Clus_Dens':gl.SArray(dtype = float),'Treatment':gl.SArray(dtype = str), 'File':gl.SArray(dtype = str),
                   'Subject':gl.SArray(dtype = str),'% Tot Clus':gl.SArray(dtype = float),'Total Area':gl.SArray(dtype = float),
                   'Total Clustered':gl.SArray(dtype = float)})
for f in os.listdir('Cluster_Data'):     #loads up every txt file in the cluster_data dir but sources from main dir
    Total = Total.append(Cluster_Volume(f))    #makes master list
    Total = Total.sort(['Subject',"Treatment"])
    Total.export_csv(filename = 'Size.txt', delimiter = '	')



Finished parsing file C:\Users\jdr248\160521 ++ V 003.txt

Parsing completed. Parsed 100 lines in 0.032003 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,str,str,str,float,str,str,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\160521 ++ V 003.txt

Parsing completed. Parsed 4749 lines in 0.039003 secs.

...

+-----------------------------+

| 20433                       |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.
